In [1]:
!pip install splink

  Using cached splink-4.0.5-py3-none-any.whl.metadata (12 kB)
  Using cached duckdb-1.1.3-cp312-cp312-win_amd64.whl.metadata (781 bytes)
  Using cached igraph-0.11.8-cp39-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached sqlglot-25.29.0-py3-none-any.whl.metadata (19 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
Using cached splink-4.0.5-py3-none-any.whl (3.7 MB)
Using cached duckdb-1.1.3-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached igraph-0.11.8-cp39-abi3-win_amd64.whl (2.0 MB)
Using cached sqlglot-25.29.0-py3-none-any.whl (426 kB)
Using cached texttable-1.7.0-py2.py3-none-any.whl (10 kB)


In [7]:
from splink import splink_datasets

df = splink_datasets.fake_1000
df = df.drop(columns=["cluster"])
df.head(5)

,unique_id,first_name,surname,dob,city,email
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net
2,2,Rob,Allen,1971-06-24,London,roberta25@smith.net
3,3,Robert,Alen,1971-06-24,Lonon,NaN
4,4,Grace,NaN,1997-04-26,Hull,grace.kelly52@jones.com


# Completeness Chart
- percentage of missing values per column can be found

In [9]:
from splink.exploratory import completeness_chart
from splink import DuckDBAPI

db_api = DuckDBAPI()
completeness_chart(df, db_api=db_api)

alt.LayerChart(...)

# EDA of data

In [14]:
from splink.exploratory import profile_columns
profile_columns(df, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

There is strong skew in the city field with around 20% of the values being London. We therefore will probably want to use term_frequency_adjustments in our linkage model, so that it can weight a match on London differently to a match on, say, Norwich.

Looking at the "Bottom 5 values by value count", we can see typos in the data in most fields. This tells us this information was possibly entered by hand, or using Optical Character Recognition, giving us an insight into the type of data entry errors we may see.

Email is a much more uniquely-identifying field than any others, with a maximum value count of 6. It's likely to be a strong linking variable.

# Blocking Rules
For most large datasets, it is computationally intractable to compare every row with every other row, since the number of comparisons rises quadratically with the number of records.

Instead we rely on blocking rules, which specify which pairwise comparisons to generate. For example, we could generate the subset of pairwise comparisons where either first name or surname matches.

This is part of a two step process to link data:

Use blocking rules to generate candidate pairwise record comparisons

Use a probabilistic linkage model to score these candidate pairs, to determine which ones should be linked

Blocking rules are the most important determinant of the performance of your linkage job.

When deciding on your blocking rules, you're trading off accuracy for performance:

If your rules are too loose, your linkage job may fail.
If they're too tight, you may miss some valid links.

In [41]:
from splink import block_on
blocked_object = block_on("first_name", "surname")

Basically you dont link records if these rules are matched. Above one could be a bit aggresive leading to miss spelling mistakes or OCR issues. So, we can start with something like below

In [46]:
block_on("substr(first_name, 1, 2)", "surname") #block based on initial of first name and surname

# Devising effective blocking rules for prediction
The aims of your blocking rules are twofold:

Eliminate enough non-matching comparison pairs so your record linkage job is small enough to compute
Eliminate as few truly matching pairs as possible (ideally none)
It is usually impossible to find a single blocking rule which achieves both aims, so we recommend using multiple blocking rules.

When we specify multiple blocking rules, Splink will generate all comparison pairs that meet any one of the rules.

For example, consider the following blocking rule:

### block_on("first_name", "dob")

This rule is likely to be effective in reducing the number of comparison pairs. It will retain all truly matching pairs, except those with errors or nulls in either the first_name or dob fields.

Now consider a second blocking rule:

### block_on("email").

This will retain all truly matching pairs, except those with errors or nulls in the email column.

Individually, these blocking rules are problematic because they exclude true matches where the records contain typos of certain types. But between them, they might do quite a good job.

For a true match to be eliminated by the use of these two blocking rules, it would have to have an error in both <b>email AND (first_name or dob)</b>.

This is not completely implausible, but it is significantly less likely than if we'd used a single rule.

More generally, we can often specify multiple blocking rules such that it becomes highly implausible that a true match would not meet at least one of these blocking criteria. This is the recommended approach in Splink. Generally we would recommend between about 3 and 10, though even more is possible.

The question then becomes how to choose what to put in this list.

# Count comparisons
- It is better to know how many comparisons will be done post applying a blocking rule

In [52]:
from splink.blocking_analysis import count_comparisons_from_blocking_rule

blocking_rule = block_on("substr(first_name, 1,1)", "surname")

counts = count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=blocking_rule,
    link_type="dedupe_only",
    db_api=db_api,
)

counts

{'number_of_comparisons_generated_pre_filter_conditions': 1632,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 473,
 'filter_conditions_identified': '',
 'equi_join_conditions_identified': 'SUBSTRING(l.first_name, 1, 1) = SUBSTRING(r.first_name, 1, 1) AND l."surname" = r."surname"',
 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}

In [54]:
# Example 2
blocking_rule = "l.first_name = r.first_name and levenshtein(l.surname, r.surname) < 2"

counts = count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule= blocking_rule,
    link_type="dedupe_only",
    db_api=db_api,
)
counts

{'number_of_comparisons_generated_pre_filter_conditions': 4827,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 372,
 'filter_conditions_identified': 'LEVENSHTEIN(l.surname, r.surname) < 2',
 'equi_join_conditions_identified': 'l.first_name = r.first_name',
 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}

# Comparison counts & best practices
The maximum number of comparisons that you can compute will be affected by your choice of SQL backend, and how powerful your computer is.

For linkages in DuckDB on a standard laptop, we suggest using blocking rules that create no more than about <b>20 million comparisons</b>. For Spark and Athena, try starting with fewer than <b>100 million comparisons</b>, before scaling up.

# Finding 'worst offending' values for your blocking rule
Blocking rules can be affected by skew: some values of a field may be much more common than others, and this can lead to a disproportionate number of comparisons being generated.

It can be useful to identify whether your data is afflicted by this problem.

In [62]:
from splink.blocking_analysis import n_largest_blocks

result = n_largest_blocks(
    table_or_tables=df,
    blocking_rule=block_on("city", "first_name"),
    link_type="dedupe_only",
    db_api=db_api,
    n_largest=3
)
result.as_pandas_dataframe()

,key_0,key_1,count_l,count_r,block_count
0,London,Oliver,7,7,49
1,Birmingham,Theodore,7,7,49
2,London,James,6,6,36


In this case, we can see that Olivers in London will result in 49 comparisons being generated. This is acceptable on this small dataset, but on a larger dataset, Olivers in London could be responsible for many million comparisons.

# Counting the number of comparisons created by a list of blocking rules
As noted above, it's usually a good idea to use multiple blocking rules. It's therefore useful to know how many record comparisons will be generated when these rules are applied.

Since the same record comparison may be created by several blocking rules, and Splink automatically deduplicates these comparisons, we cannot simply total the number of comparisons generated by each rule individually.

Splink provides a chart that shows the marginal (additional) comparisons generated by each blocking rule, after deduplication

In [67]:
from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart

blocking_rules_for_analysis = [
    block_on("substr(first_name, 1,1)", "surname"),
    block_on("surname"),
    block_on("email"),
    block_on("city", "first_name"),
    "l.first_name = r.first_name and levenshtein(l.surname, r.surname) < 2",
]


cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rules=blocking_rules_for_analysis,
    db_api=db_api,
    link_type="dedupe_only",
)

alt.Chart(...)

Above graph talks about the comparisons added by each blocking rule and what are the total comparisons with these blocking rules
- 1 - 473 
- 2 - 1165
- 3 - 468
- 4 - 145
- 5 - 30 (This if used in isolation generates 372 as found in above cells)

# Digging deeper: Understanding why certain blocking rules create large numbers of comparisons
Finally, we can use the profile_columns function we saw in the previous tutorial to understand a specific blocking rule in more depth.

Suppose we're interested in blocking on city and first initial.

Within each distinct value of (city, first initial), all possible pairwise comparisons will be generated.

So for instance, if there are 15 distinct records with London,J then these records will result in n(n-1)/2 = 105 pairwise comparisons being generated.

In a larger dataset, we might observe 10,000 London,J records, which would then be responsible for 49,995,000 comparisons.

These high-frequency values therefore have a disproportionate influence on the overall number of pairwise comparisons, and so it can be useful to analyse skew, as follows:

In [74]:
profile_columns(df, column_expressions=["city || left(first_name,1)"], db_api=db_api)

alt.VConcatChart(...)

These comparison counts help us analyze how to distribute the load to spark clusters in case of large data sets

Now it's time to estimate a probabilistic linkage model to score each of these comparisons. The resultant match score is a prediction of whether the two records represent the same entity (e.g. are the same person).

The purpose of estimating the model is to learn the relative importance of different parts of your data for the purpose of data linking.

For example, a match on date of birth is a much stronger indicator that two records refer to the same entity than a match on gender. A mismatch on gender may be a stronger indicate against two records referring than a mismatch on name, since names are more likely to be entered differently.

The relative importance of different information is captured in the (partial) 'match weights', which can be learned from your data. These match weights are then added up to compute the overall match score.

The match weights are are derived from the m and u parameters of the underlying Fellegi Sunter model. Splink uses various statistical routines to estimate these parameters. Further details of the underlying theory can be found https://www.robinlinacre.com/intro_to_probabilistic_linkage/ , which will help you understand this part of the tutorial.

# Some good points from the link above
- Record linkage and deduplication are equivalent problems. The only difference is that linkage involves finding matching entities across datasets and deduplication involves finding matches within datasets
- For fuzzy match of firstnames use Jaro Winkler similarity, more at https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance#Jaro%E2%80%93Winkler_similarity
- Levenshtein better for DOB which calculates min number of single character edits needed for one string to become other, more at https://en.wikipedia.org/wiki/Levenshtein_distance
- probability= 2^w/(1+2^w) where w is the final weight of the match 
- There's a one to one correspondence between columns and partial match weights. Each column is treated as a separate piece of information. In reality, the model allows for more flexibility. We could split a column (e.g. date of birth) into several pieces of information (e.g. day, month, year) and estimate partial match weights for each. Or we could combine several columns (e.g. first, middle and surname) into a single piece of information and estimate a partial match weights for this information
- m is the prob how often scenario occurs among matching records
- u is the prob how often scenario doesnt occurs non-matching records
- Bayes Factor = K = m/u
- A Bayes Factor of 5 can be interpreted as '5 times more likely to match
- A Bayes Factor of 0.2 can be interpreted as '5 times less likely to match
- Example-1
  - Amongst matching records, month of birth will usually match. Supposing the occasional typo, we may have m=0.99
  - Amongst non matching records, month of birth matches around a twelth of the time, so u=1/12
  - K = m/u = 11.9
  - This means we observe this scenario around 11.9 times more often amongst matching records than non-matching records.
  - Hence, given this observation, the records are 11.9 times more likely to be a match.
  - More generally, we can see from the formula that strong positive match weights only possible with low u probabilities, implying high cardinality.
- Example-2
  - Amongst matching records, it will be rare to observe a non-match on gender. If there are occasional data entry errors, we may have m=0.02
  - Amongst non matching records, gender will match around half the time. So u=0.5.
  - K = 1/25
  - We observe this scenario around 25 times more often among non-matching records than matching records.
  - Hence, given this observation the records are 25 times less likely to be a match.
  - More generally, we can see from the formula that strong negative match weights only possible with low m probabilities, which in turn implies high data quality.
- m
  - The m probability can be thought of as a measure of data quality, or the propensity for data to change through time.
  - An m probability of 0.9 means that, amongst matching records, the first name matches just 90% of the time, which is an indication of poor data quality.
- u
  - The u probability is primarily a measure of the likelihood of coincidences, which is driven by the cardinality of the data.
  - A u probability of 0.005 means that, amongst non-matching records, first name matches 0.5% of the time.
  - The u probability therefore measures how often two different people have the same first name - so in this sense it's a measure of how often coincidences occur.
- It's only meaningful to say that something is more or less likely relative to a starting probability - known as the 'prior' (our 'prior belief').
- In the context of record linkage, the prior is our existing belief that the two records match before we saw the new information contained in a scenario (in RANDOM)
- we go with partial match weight over actual weight to bring the values to a range of -10 to 30 where w=log2(m/u) or log2(K)

In [82]:
import splink.comparison_library as cl

city_comparison = cl.LevenshteinAtThresholds("city", 2)
print(city_comparison.get_comparison("duckdb").human_readable_description)

Comparison 'LevenshteinAtThresholds' of "city".
Similarity is assessed using the following ComparisonLevels:
    - 'city is NULL' with SQL rule: "city_l" IS NULL OR "city_r" IS NULL
    - 'Exact match on city' with SQL rule: "city_l" = "city_r"
    - 'Levenshtein distance of city <= 2' with SQL rule: levenshtein("city_l", "city_r") <= 2
    - 'All other comparisons' with SQL rule: ELSE



In [84]:
email_comparison = cl.EmailComparison("email")
print(email_comparison.get_comparison("duckdb").human_readable_description)

Comparison 'EmailComparison' of "email".
Similarity is assessed using the following ComparisonLevels:
    - 'email is NULL' with SQL rule: "email_l" IS NULL OR "email_r" IS NULL
    - 'Exact match on email' with SQL rule: "email_l" = "email_r"
    - 'Exact match on username' with SQL rule: NULLIF(regexp_extract("email_l", '^[^@]+', 0), '') = NULLIF(regexp_extract("email_r", '^[^@]+', 0), '')
    - 'Jaro-Winkler distance of email >= 0.88' with SQL rule: jaro_winkler_similarity("email_l", "email_r") >= 0.88
    - 'Jaro-Winkler >0.88 on username' with SQL rule: jaro_winkler_similarity(NULLIF(regexp_extract("email_l", '^[^@]+', 0), ''), NULLIF(regexp_extract("email_r", '^[^@]+', 0), '')) >= 0.88
    - 'All other comparisons' with SQL rule: ELSE



In [92]:
from splink import Linker, SettingsCreator, block_on, DuckDBAPI

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        cl.LevenshteinAtThresholds("dob", 1),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "city"),
        block_on("surname"),

    ],
    retain_intermediate_calculation_columns=True,
)

linker = Linker(df, settings, db_api=DuckDBAPI())

We are performing a dedupe_only (the other options are link_only, or link_and_dedupe, which may be used if there are multiple input datasets).
When comparing records, we will use information from the first_name, surname, dob, city and email columns to compute a match score.
The blocking_rules_to_generate_predictions states that we will only check for duplicates amongst records where either the first_name AND city or surname is identical.
We have enabled term frequency adjustments for the 'city' column, because some values (e.g. London) appear much more frequently than others.
We have set retain_intermediate_calculation_columns and additional_columns_to_retain to True so that Splink outputs additional information that helps the user understand the calculations. If they were False, the computations would run faster.

In [97]:
# calculating prior
deterministic_rules = [
    block_on("first_name", "dob"),
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email")
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00298.
This means that amongst all possible pairwise record comparisons, one in 335.56 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,488.57 matching pairs


In [99]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [101]:
linker.estimate_m_from_label_column("social_security_number") # but we dont have the truth

AttributeError: 'Linker' object has no attribute 'estimate_m_from_label_column'

All algos to calculate m are - https://moj-analytical-services.github.io/splink/api_docs/training.html

In [103]:
training_blocking_rule = block_on("first_name", "surname")
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.521 in the m_probability of dob, level `Exact match on dob`
Iteration 2: Largest change in params was 0.0516 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0183 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00744 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00349 in probability_two_random_records_match
Iteration 6: Larg

In [105]:
training_blocking_rule = block_on("dob")
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.407 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.0929 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0548 in the m_probability of first_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.0186 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00758 in probability_two_random_records_match
Iteration 6: Largest change in p

In [107]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

In [109]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

In [111]:
linker.visualisations.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [116]:
settings = linker.misc.save_model_to_json(
    "./saved_model.json", overwrite=True
)

# Detecting unlinkable record
An interesting application of our trained model that is useful to explore before making any predictions is to detect 'unlinkable' records.

Unlinkable records are those which do not contain enough information to be linked. A simple example would be a record containing only 'John Smith', and null in all other fields. This record may link to other records, but we'll never know because there's not enough information to disambiguate any potential links. Unlinkable records can be found by linking records to themselves - if, even when matched to themselves, they don't meet the match threshold score, we can be sure they will never link to anything

In [119]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)

In the above chart, we can see that about 1.3% of records in the input dataset are unlinkable at a threshold match weight of 6.11 (correponding to a match probability of around 98.6%)
for 98.6% prob, the w is 6.11 i.e. 1.3% of records are unlinkables

In [128]:
import json

file = "./saved_model.json"

with open(file, 'r') as file_object :
    settings = json.load(file_object)


linker = Linker(df, settings, db_api=DuckDBAPI())

In [130]:
df_predictions = linker.inference.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe(limit=5)

Blocking time: 0.01 seconds
Predict time: 0.20 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,tf_email_l,tf_email_r,bf_email,bf_tf_adj_email,match_key
0,6.311409,0.987565,8,9,None,Evie,-1,NaN,0.008424,1.000000,...,1.000000,1.000000,None,evihd56@earris-bailey.net,-1,NaN,0.001267,1.000000,1.000000,1
1,18.868966,0.999998,26,28,Thomas,Thomas,4,0.006017,0.006017,84.821765,...,0.462956,1.000000,gabriel.t54@nnichls.info,gabriel.t54@nichols.info,3,0.001267,0.002535,212.576644,1.000000,1
2,10.984407,0.999507,29,30,Thomas,Thomas,4,0.006017,0.006017,84.821765,...,1.000000,1.000000,None,gabriel.t54@nlchois.info,-1,NaN,0.001267,1.000000,1.000000,1
3,21.210697,1.000000,37,39,Theodore,Theodore,4,0.012034,0.012034,84.821765,...,10.201260,1.120874,t.m39@brooks-sawyer.com,t.m39@brooks-sawyer.com,4,0.006337,0.006337,252.050601,0.346193,0
4,1.212079,0.698497,42,43,Theodore,Theodore,4,0.012034,0.012034,84.821765,...,0.462956,1.000000,None,t.m39@brooks-sawyer.com,-1,NaN,0.006337,1.000000,1.000000,1


In [134]:
pd_df_perdictions = df_predictions.as_pandas_dataframe()

In [138]:
pd_df_perdictions.match_key.unique()

array(['1', '0'], dtype=object)

In [140]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predictions, threshold_match_probability=0.5
)
clusters.as_pandas_dataframe(limit=10)

Completed iteration 1, num representatives needing updating: 2
Completed iteration 2, num representatives needing updating: 0


,cluster_id,unique_id,first_name,surname,dob,city,email
0,8,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net
1,14,14,Oliver,Griffiths,1991-10-26,Lunton,o.griffiths90@reyes-coleman.com
2,22,24,Thoas,Green,1974-10-05,London,thomas.green@clark.org
3,26,26,Thomas,Gabriel,1976-09-15,Loodon,gabriel.t54@nnichls.info
4,26,30,Thomas,Gabriel,1976-09-15,London,gabriel.t54@nlchois.info
5,37,37,Theodore,Morris,1978-08-19,Birmingham,t.m39@brooks-sawyer.com
6,37,39,Theodore,Morris,1978-08-19,Birmingham,t.m39@brooks-sawyer.com
7,37,43,Theodore,Morris,1978-08-19,Birmingham,t.m39@brooks-sawyer.com
8,52,52,Jyayden,Bnennet,2017-01-11,Snawseaa,jb88@king.com
9,74,74,Ronni,Begum,2003-10-15,London,r.b80@ellis-berry.com


In [144]:
sql = f"""
select *
from {df_predictions.physical_name}
limit 2
"""
misc_df = linker.misc.query_sql(sql)

In [146]:
misc_df

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,tf_email_l,tf_email_r,bf_email,bf_tf_adj_email,match_key
0,6.311409,0.987565,8,9,None,Evie,-1,NaN,0.008424,1.000000,...,1.000000,1.0,None,evihd56@earris-bailey.net,-1,NaN,0.001267,1.000000,1.0,1
1,18.868966,0.999998,26,28,Thomas,Thomas,4,0.006017,0.006017,84.821765,...,0.462956,1.0,gabriel.t54@nnichls.info,gabriel.t54@nichols.info,3,0.001267,0.002535,212.576644,1.0,1


In [148]:
records_to_view = df_predictions.as_record_dict(limit=5)
linker.visualisations.waterfall_chart(records_to_view, filter_nulls=False)

alt.LayerChart(...)

In [150]:
linker.visualisations.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame

IFrame(src="./scv.html", width="100%", height=1200)

In [152]:
df_clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predictions, threshold_match_probability=0.5
)

linker.visualisations.cluster_studio_dashboard(
    df_predictions,
    df_clusters,
    "cluster_studio.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame

IFrame(src="./cluster_studio.html", width="100%", height=1000)

Completed iteration 1, num representatives needing updating: 2
Completed iteration 2, num representatives needing updating: 0


In [154]:
settings["blocking_rules_to_generate_predictions"] = [
    block_on("first_name"),
    block_on("city"),
    block_on("email"),
    block_on("dob"),
]

linker = Linker(df, settings, db_api=DuckDBAPI())
df_predictions = linker.inference.predict(threshold_match_probability=0.01)

Blocking time: 0.01 seconds
Predict time: 0.64 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


In [158]:
from splink.datasets import splink_dataset_labels
df_labels = splink_dataset_labels.fake_1000_labels
labels_table = linker.table_management.register_labels_table(df_labels)
df_labels.head(5)

downloading: https://raw.githubusercontent.com/moj-analytical-services/splink_datasets/master/data/fake_1000_labels.csv



,unique_id_l,source_dataset_l,unique_id_r,source_dataset_r,clerical_match_score
0,0,fake_1000,1,fake_1000,1.0
1,0,fake_1000,2,fake_1000,1.0
2,0,fake_1000,3,fake_1000,1.0
3,0,fake_1000,4,fake_1000,0.0
4,0,fake_1000,5,fake_1000,0.0


In [160]:
splink_df = linker.evaluation.prediction_errors_from_labels_table(
    labels_table, include_false_negatives=True, include_false_positives=False
)
false_negatives = splink_df.as_record_dict(limit=5)
linker.visualisations.waterfall_chart(false_negatives)

alt.LayerChart(...)

In [162]:
splink_df = linker.evaluation.prediction_errors_from_labels_table(
    labels_table, include_false_negatives=False, include_false_positives=True, threshold_match_probability=0.01
)
false_postives = splink_df.as_record_dict(limit=5)
linker.visualisations.waterfall_chart(false_postives)

alt.LayerChart(...)

In [166]:
linker.evaluation.accuracy_analysis_from_labels_table(
    labels_table, output_type="threshold_selection", add_metrics=["f1"]
)

alt.HConcatChart(...)

In [168]:
linker.evaluation.accuracy_analysis_from_labels_table(labels_table, output_type="roc")

alt.Chart(...)

In [170]:
roc_table = linker.evaluation.accuracy_analysis_from_labels_table(
    labels_table, output_type="table"
)
roc_table.as_pandas_dataframe(limit=5)

,truth_threshold,match_probability,total_clerical_labels,p,n,tp,tn,fp,fn,P_rate,...,precision,recall,specificity,npv,accuracy,f1,f2,f0_5,p4,phi
0,-18.9,0.000002,3176.0,2031.0,1145.0,1709.0,1103.0,42.0,322.0,0.639484,...,0.976014,0.841457,0.963319,0.774035,0.885390,0.903755,0.865316,0.945766,0.880476,0.776931
1,-16.7,0.000009,3176.0,2031.0,1145.0,1709.0,1119.0,26.0,322.0,0.639484,...,0.985014,0.841457,0.977293,0.776544,0.890428,0.907594,0.866721,0.952514,0.886010,0.789637
2,-12.8,0.000140,3176.0,2031.0,1145.0,1709.0,1125.0,20.0,322.0,0.639484,...,0.988433,0.841457,0.982533,0.777471,0.892317,0.909043,0.867249,0.955069,0.888076,0.794416
3,-12.5,0.000173,3176.0,2031.0,1145.0,1708.0,1125.0,20.0,323.0,0.639484,...,0.988426,0.840965,0.982533,0.776934,0.892003,0.908752,0.866829,0.954937,0.887763,0.793897
4,-12.4,0.000185,3176.0,2031.0,1145.0,1705.0,1132.0,13.0,326.0,0.639484,...,0.992433,0.839488,0.988646,0.776406,0.893262,0.909576,0.866186,0.957542,0.889225,0.797936
